In [1]:
import os
import pickle
import random
import numpy as np
import pandas as pd
# import cv2
# import matplotlib
# matplotlib.use('Agg')  # Use non-interactive backend
# import matplotlib.pyplot as plt
# import matplotlib.animation as animation
# from matplotlib.lines import Line2D
# from scipy.interpolate import interp1d
# import subprocess

from utils_pipeline.pipeline_B import *
from utils_mocap_viz.video_layout import combine_views, get_video_paths

### Prepare window around beats and subdivision

In [2]:
traj_dir  = "traj_files"
file_name = "BKO_E1_D1_02_Maraka"
status    = "included"   # or "excluded"
value     = "0.4"        # or any other threshold

cycle_segs, windows = compute_windows(traj_dir, file_name, status, value)

Loaded cycle_segments from BKO_E1_D1_02_Maraka_included_0.4.pkl
cycle_segments: [(166.55722222199998, 168.02211111066666), (168.02211111066666, 169.503), (169.503, 170.9785555553333), (170.9785555553333, 172.4505555553333), (172.4505555553333, 173.9083333333333), (173.9083333333333, 175.364333333), (175.364333333, 176.802555555), (176.802555555, 178.222111111), (198.59099999966668, 199.94477777766667), (328.99099999966666, 330.274555555), (330.274555555, 331.5225555553333), (331.5225555553333, 332.758111111), (332.758111111, 333.9989999996667), (333.9989999996667, 335.2452222223333), (335.2452222223333, 336.5394444445)]
Number of windows for each beat/subdivision:
  beat_1: 15 windows
  beat_2: 15 windows
  beat_3: 15 windows
  beat_4: 15 windows
  subdiv_2: 15 windows
  subdiv_3: 15 windows
  subdiv_5: 15 windows
  subdiv_6: 15 windows
  subdiv_8: 15 windows
  subdiv_9: 15 windows
  subdiv_11: 15 windows
  subdiv_12: 15 windows


### Generate trajectory video + dance video plots

In [3]:
w_key = "subdiv_2"
vid_plot_path = f"cycle_videos/{file_name}/{w_key}/"

traj_tuples = random.sample(windows[w_key], 2)  # Randomly sample 2 tuples from the list

# traj_tuples = windows[w_key][0:nb]
print(windows.keys())
print(len(windows[w_key]))

dict_keys(['beat_1', 'beat_2', 'beat_3', 'beat_4', 'subdiv_2', 'subdiv_3', 'subdiv_5', 'subdiv_6', 'subdiv_8', 'subdiv_9', 'subdiv_11', 'subdiv_12'])
15


In [ ]:
extract_cycle_videos_and_plots(
    file_name = file_name,
    windows = traj_tuples,  # List of (win_start, win_end, t_poi) tuples
    window_key = w_key,
    figsize = (10, 3),
    dpi = 200,
    )

### Build concatenated videos

In [ ]:
concatenate_and_overlay_videos(file_name, vid_plot_path)

### Combine Videos

In [4]:
concat_plot = os.path.join(vid_plot_path, "concatenated_plot.mp4")  # size 2000 by 600
concat_video = os.path.join(vid_plot_path, "concatenated_video.mp4") # 1920 by 1080

view_videos = {
    'concatenated_video': concat_video,  # Your concatenated video path
    'concatenated_plot': concat_plot,    # Your concatenated plot path
}

layout = [
    {'view': 'concatenated_video', 'x': 0, 'y': 0, 'width': 1920, 'height': 540},
    {'view': 'concatenated_plot', 'x': 0, 'y': 540, 'width': 1920, 'height': 540}
]

final_output = combine_trajectory_views(
    filename= file_name,
    output_dir= vid_plot_path,
    view_videos= view_videos,
    layout_config= layout,
    video_size= (1920, 1280),
    fps=24
)


Video 1 info:
{
    "programs": [

    ],
    "streams": [
        {
            "width": 1920,
            "height": 1080,
            "r_frame_rate": "50/1"
        }
    ]
}


Video 2 info:
{
    "programs": [

    ],
    "streams": [
        {
            "width": 2000,
            "height": 600,
            "r_frame_rate": "50/1"
        }
    ]
}


Preparing to combine videos:
Output file: cycle_videos/BKO_E1_D1_02_Maraka/subdiv_2/final_combined_BKO_E1_D1_02_Maraka.mp4
Frame rate: 24 fps
Number of input files: 2
Video dimensions: 1920x540 for each video
Input video 1: cycle_videos/BKO_E1_D1_02_Maraka/subdiv_2/concatenated_video.mp4
Input video 2: cycle_videos/BKO_E1_D1_02_Maraka/subdiv_2/concatenated_plot.mp4

FFmpeg command:
ffmpeg -y -i cycle_videos/BKO_E1_D1_02_Maraka/subdiv_2/concatenated_video.mp4 -i cycle_videos/BKO_E1_D1_02_Maraka/subdiv_2/concatenated_plot.mp4 -filter_complex [0:v]scale=1920:540:force_original_aspect_ratio=1,pad=1920:540:(ow-iw)/2:(oh-ih)/2[v0];[1:v]scal